# Tutorial 1 - Neural network intro

In this tutorial, we'll introduce the **Python** programming language and some of its libraries, most notably [PyTorch](https://pytorch.org/).

The approaches used by PyTorch are broadly applicable to other frameworks as well, such as [TensorFlow](https://www.tensorflow.org/), [JAX](https://docs.jax.dev/en/latest/), [Flux.jl](https://fluxml.ai/) or [Deep Learning Toolbox](https://www.mathworks.com/products/deep-learning.html).

For a more detailed overview of PyTorch you can also go through the [oficial tutorials](https://pytorch.org/tutorials/).

## Package installation with uv and Jupyter

We'll first install the [uv](https://docs.astral.sh/uv/) package manager. We can install Jupyter system-wide with uv by running:

```bash
$ uv tool install jupyterlab
```

We need to create a uv project and connect it to Jupyter:
```bash
$ uv init
$ uv add --dev ipykernel
$ uv run ipython kernel install --user --env VIRTUAL_ENV $(pwd)/.venv --name=tzn
```

In order for the newly installed Jupyter kernel to be available, we need to restart Jupyter.

Finally, we can install the required packages:
```bash
$ uv add matplotlib numpy scikit-learn torch torch-geometric torchvision
$ uv add torch-cluster -f https://data.pyg.org/whl/torch-2.8.0+cpu.html
```

## Task 1

Run the aforementioned steps to create a package and have the next cell run without errors. It should print something like `Torch version: 2.8.0`.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

print("Torch version:", torch.__version__)

## Tensors and working with them in PyTorch

Let's first take a look at how to create and manipulate tensors and what is happening in the background in order to automatically compute gradients. 

A tensor is in essence a name for an array with $n$ dimentisons - i. e. like the `ndarray` in numpy.

We can create a PyTorch tensor by converting from python or numpy:

In [ ]:
a = torch.tensor([[1, 2], [3,4]])
print(a)

We can also convert it back to numpy:

In [ ]:
print(a.numpy())

A tensor always has the attributes `shape` and `dtype`:

In [ ]:
print('Shape:', a.shape)
print('Element data type:', a.dtype)

Other ways of constructing a tensor:

In [ ]:
print(torch.ones((1, 3)))
print(torch.zeros((1, 3)))
print(torch.randn((1, 4))) # Normal distribution
print(torch.rand((1, 4))) # Uniform distribution on [0, 1)

We can do basic operations with PyTorch tensors. Let's construct three tensors with the same dtype:

In [ ]:
a = torch.tensor([[1, 2], [3,4]], dtype = torch.float32)
b = torch.ones((2, 1))
c = torch.randn((2, 1))
print(a)
print(b)
print(c)

Matrix multiplication:

In [ ]:
print(a.matmul(b))

Element-wise addition:

In [ ]:
print(b.add(c))

Element-wise square:

In [ ]:
print(a.square())

Euclidean norm of a vector:

In [ ]:
print(b.norm())

Matrix rank:

In [ ]:
print(torch.linalg.matrix_rank(a))

Element-wise application of the sine function:

In [ ]:
print(torch.sin(a))

Boolean indicator of whether each element is greater than 1:

In [ ]:
print(a.greater(1))

## Task 2 - basics of working with data

Let's load the [MNIST](https://en.wikipedia.org/wiki/MNIST_database) dataset and calculate the average value of each feature. Running the next cell for the first time downloads the dataset and may take a while.

In [ ]:
data = datasets.MNIST(
    root = "data",
    download = True,
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(0, 1)])
)

We've loaded images from the MNIST dataset. Now try to calculate the average value of each feature accross all images. Use the [`DataLoader`](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) class.

## Automatic gradient calculation

To automatically calculate gradients, we use the regular `torch.Tensor`. To calculate gradients with respect to some variable, we need to set it to `requires_grad = True`. This will make PyTorch track all operations which depend on this variable. To actually calculate the gradient, we need to call the `fun.backward()` method, which will calculate th derivative of `fun` with respect to all the input variables. If `fun` is of greater dimension than 1, we need to pass to `backward` a tensor with repspect to which to calculate the Jacobian.

In [ ]:
x = torch.tensor([1, 2, 3], dtype = torch.float32, requires_grad = True)
print(x)

In [ ]:
f = torch.square(x)
f.backward(torch.ones_like(f))

print('Derivative of x^2:', x.grad)
with torch.no_grad():
    print('2*x:', 2*x)

In [ ]:
x = torch.tensor([1, 2, 3], dtype = torch.float32, requires_grad = True)
print(x)

In [ ]:
f = torch.sin(x)
f.backward(torch.ones_like(f))
    
print('Derivative of sin(x):', x.grad)
with torch.no_grad():
    print('cos(x):', torch.cos(x))

## Basic neural net on the MNIST dataset

Let's define the basic parameters of the model:

In [ ]:
batch_size = 50
hidden_layer_width = 100
output_width = 10
learning_rate = 0.01

Let's load the training and testing data and create a `DataLoader` for each:

In [ ]:
data_train = datasets.MNIST(
    root = "data",
    train = True,
    download = True,
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(0, 1)])
)
data_test = datasets.MNIST(
    root = "data",
    train = False,
    download = True,
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(0, 1)])
)

dataloader_train = DataLoader(data_train, batch_size = batch_size, shuffle = True)
dataloader_test = DataLoader(data_test, batch_size = batch_size, shuffle = True)

We'll create a simple neural network using `torch.nn.Sequential`, which is a class to put several layers into sequentially in the order of their insertion.

The input layer will be a `torch.nn.Flatten`, which will convert the 2D images into vectors.

`torch.nn.Sequential` is a specialization of the more general [`tf.nn.Module`](https://pytorch.org/docs/main/generated/torch.nn.Module.html) class.

In [ ]:
model = nn.Sequential()
model.append(nn.Flatten())

Let's add one hidden layer with the `tanh` activation function:

In [ ]:
model.append(nn.Linear(data_train.data.shape[1] * data_train.data.shape[2], hidden_layer_width))
model.append(nn.Tanh())

Finally, let's add an output layer with 10 neurons (= number of classes) and the softmax activation function: 
$$ \sigma \left( \vec{z} \right)_i = \frac{e^{\vec{z}_i}}{\sum_{j = 1}^K e^{\vec{z}_j}} $$

In [ ]:
model.append(nn.Linear(hidden_layer_width, output_width))
model.append(nn.Softmax(dim=1))

Printing a model will also give us a summary:

In [ ]:
model

Such a model may also be directly applied to data liek a function.

Its outputs will be probability vectors. Because we haven't trained the model yet, the results will be random.

In [ ]:
model(data_train[0][0])

The final prediction of the model could be realized e.g. as the element with the highest probability:

In [ ]:
plt.figure(figsize=(10, 7))
for i in range(24):
    plt.subplot(4, 6, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(data_train[i][0].reshape(28, 28), cmap = plt.cm.bone)
    plt.title(model(data_train[i][0]).detach().argmax(axis = 1).item())

## Training the model

Now we can actually train the model. We'll need two things for that:
* The *loss* function - the function that we want to minimize
* The *optimizer* - a function that uses the gradient to make one optimization step

Note:
* `torch.nn.NLLLoss` is the cross-entropy loss function which takes as input a probability distribution
* `torch.nn.CrossEntropyLoss` is the cross-netropy loss function, which only takes as inputs class weights (i. e. they don't need to sum up to 1)

In other words, `CrossEntropyLoss` combines `NLLLoss` and `Softmax`.

We will use the ADAM optimizer.

In [ ]:
loss_fn = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters())
epochs = 10

To train the model we will use the `DataLoader`. We will train in mini-batches, which means that we will split the datset into smaller batches, calculate the average loss function for the whole batch and then use it to make one optimization step. One epoch then means using the whole training dataset once (which includes many optimization steps).

Let us define 3 functions needed to train the model:

In [ ]:
def calculate_accuracy(model, dataloader):
    num_correct = 0
    
    with torch.no_grad():
        for (X, y) in dataloader:
            pred = model(X)
            num_correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    accuracy = num_correct / len(dataloader.dataset)
    return accuracy

In [ ]:
def calculate_loss_accuracy(model, dataloader, loss_fn):
    loss = 0
    num_correct = 0
    
    with torch.no_grad():
        for (X, y) in dataloader:
            pred = model(X)
            loss += loss_fn(pred, y).item()
            num_correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    loss /= len(dataloader)
    accuracy = num_correct / len(dataloader.dataset)
    return loss, accuracy

In [ ]:
def train_model(model, loss_fn, optimizer, epochs, dataloader_train, dataloader_test, early_stopper = None, log_period = 10000):
    for epoch in range(epochs):
        processed_since_log = 0
        for batch, (X, y) in enumerate(dataloader_train):
            model.train()
            pred = model(X)
            loss = loss_fn(pred, y)

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            processed_since_log += dataloader_train.batch_size

            if processed_since_log >= log_period:
                current = min((batch + 1) * dataloader_train.batch_size, len(data_train))
                loss = loss.item()
                model.eval()
                train_acc = calculate_accuracy(model, dataloader_train)
                test_loss, test_acc = calculate_loss_accuracy(model, dataloader_test, loss_fn)
                print(f"train loss: {loss:>7f}  test loss: {test_loss:>7f}  train accuracy: {train_acc:>3f}  test accuracy: {test_acc:>3f}  [sample {current:>5d}/{len(data_train):>5d}] [epoch {epoch+1:>2d}/{epochs:>2d}]")
                processed_since_log -= log_period

In [ ]:
train_model(model, loss_fn, optimizer, epochs, dataloader_train, dataloader_test)

### Evaluating the model

To evaluate the model on the test set, we can reuse the previously defined functions.

In [ ]:
model.eval()
test_loss, test_acc = calculate_loss_accuracy(model, dataloader_test, loss_fn)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

If we need the "raw" output values, we may again just use the model as a function:

In [ ]:
predictions = model(data_test[0][0])
print('Prediction shape:', predictions.shape)
print('Probablity of the first image being a zero:', predictions[0, 0].item())

To obtain actual class predictions, we will use the `torch.argmax` function to find out which digit has the highest probability:

In [ ]:
Y_pred = predictions.detach().argmax(dim = 1)
print('Probabilities for the first image:', predictions.detach())
print('Most likely label for the first image:', Y_pred[0].item())

In [ ]:
plt.figure(figsize=(10, 7))
for i in range(24):
    plt.subplot(4, 6, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(data_test[i][0].reshape(28, 28), cmap = plt.cm.bone)
    plt.title(model(data_test[i][0]).detach().argmax(axis = 1).item())

## Task 3 - Creating the model manually

Let's again define the basic properties of the model:

In [ ]:
batch_size = 50
hidden_layer_width = 100
output_width = 10
learning_rate = 0.01
epochs = 10
loss_fn = nn.CrossEntropyLoss()

Let's define the parameters we will need for a neural network with one hidden layer with 100 neurons:

In [ ]:
W1 = (0.1 * torch.randn((data_train[0][0].shape[1] * data_train[0][0].shape[2], hidden_layer_width))).clone().requires_grad_(True)
b1 = torch.zeros((hidden_layer_width,), requires_grad = True)
W2 = (0.1 * torch.randn((hidden_layer_width, output_width))).clone().requires_grad_(True)
b2 = torch.zeros((output_width,), requires_grad = True)

Let's define the forward pass of the neural net in a function:

In [ ]:
def predict(inputs):
    # TODO

Let's train the model:

In [ ]:
log_period = 10000

for epoch in range(epochs):
    processed_since_log = 0
    for batch, (X, y) in enumerate(dataloader_train):
        pred = predict(X)
        loss = loss_fn(pred, y)
        loss.backward()

        for variable in [W1, b1, W2, b2]:
            with torch.no_grad():
                variable -= variable.grad * learning_rate
            variable.grad = None

        processed_since_log += dataloader_train.batch_size

        if processed_since_log >= log_period:
            current = min((batch + 1) * dataloader_train.batch_size, len(data_train))
            loss = loss.item()
            train_acc = calculate_accuracy(predict, dataloader_train)
            test_loss, test_acc = calculate_loss_accuracy(predict, dataloader_test, loss_fn)
            print(f"train loss: {loss:>7f}  test loss: {test_loss:>7f}  train accuracy: {train_acc:>3f}  test accuracy: {test_acc:>3f}  [sample {current:>5d}/{len(data_train):>5d}] [epoch {epoch+1:>2d}/{epochs:>2d}]")
            processed_since_log -= log_period

## Training (hyper-)parameters

In the previous examples we just set the various hyper-parameters of the network semi-randomly. Let's now take a look at them and their possible values.

### Network architecture

- Number of layers
- Layer widths
- activation functions
    - linear
    - tanh
    - sigmoid
    - hard sigmoid
    - relu
    - selu
    - softmax

In [ ]:
plt.figure(figsize=(20, 20))
i = 1
for activationFunction in [nn.Tanh(), nn.Hardtanh(), nn.Sigmoid(), nn.Hardsigmoid(), nn.ReLU(), nn.LeakyReLU(), nn.SELU(), nn.ELU()]:
    plt.subplot(4, 4, i)
    i += 1
    plt.grid(True)
    xs = torch.linspace(-5, 5, 100);
    ys = activationFunction(xs)
    plt.plot(xs, ys)
    plt.title(type(activationFunction).__name__)

## Loss functions

- Mean square error
    $$ \operatorname{MSE}=\frac{1}{n}\sum_{i=1}^n(Y_i-\hat{Y_i})^2. $$
- Hinge
    $$ \operatorname{Hinge} = \sum_{i=1}^K \max(0, 1- Y_i \cdot \hat{Y_i}) $$
- Cross-entropy
    $$ \operatorname{crossentropy} = - \sum_{i=1}^K Y_i \cdot \log(\hat{Y_i}) $$

## Optimization algorithms

- SGD
- RMSProp
- Adagrad
- Adam
- Adadelta
- Adamax
- Nadam
- AdamW

### Learning schedule

We may optionally add a so-called weight decay to the optimization algorithm. This means that the learning rate will become smaller as we train. For example:

In [ ]:
initial_learning_rate = 0.01
batch_count = 64
optimizer = torch.optim.SGD(model.parameters(), weight_decay=0.001)

More advanced algorithms such as AdamW have weight decay built-in.

## Using the training dataset

We can set 2 ways in which the training dataset is used:
- Epochs: How many times we loop over the whole dataset
- Batching: Using the dataset not sample-by-sample but in batches

# Task 4

Try to modify the previously defined model on MNIST to obtain the best possible results by tuning its hyper-parameters.